In [46]:
import bs4 as bs  
#from bs4 import BeautifulSoup as bs
import urllib.request  
import re
import nltk
import heapq
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx


In [74]:

# input WIKI URL...
def input(url):
    scraped_data = urllib.request.urlopen(url)
    article = scraped_data.read()
    parsed_article = bs.BeautifulSoup(article,'lxml')
    #print(parsed_article)
    paragraphs = parsed_article.find_all('p')
    #print(paragraphs)
    article_text = ""
    for p in paragraphs:  
        article_text += p.text
    #print(article_text)
    return(article_text)

# Sentence Tokenization
def sent_tokens(text):
    sents = nltk.sent_tokenize(text)
    #print(sents)
    return(sents)




In [100]:

def preprocess(sentences):
    sent_clean=[]
    sent_clean1=[]
    stop_words = nltk.corpus.stopwords.words('english')
    for sent in sentences:
        sent_clean = " ".join([word for word in nltk.word_tokenize(sent.lower()) if word not in stop_words])
        #print(sent_clean)
        sent_clean = [re.sub(r'[^a-zA-Z]'," ",sent_clean)]
        #print(sent_clean)
        #sent_clean = [re.sub(r'\s+', " ",sent_clean(i))]
        sent_clean1 +=sent_clean
    #print(sent_clean1)
    return(sent_clean1)

# Extract word vectors
def word_embedded(filename,cleaned_sentences,num_sentences): #file name with path
    word_embeddings = {}
    f = open(filename, encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    sentence_vectors=[]
    for i in cleaned_sentences:
        if len(i)!=0:
            v = sum([word_embeddings.get(w,np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v= np.zeros((100))
        sentence_vectors.append(v)
        
    #creating similarity_matrix
    sim_mat = np.zeros([len(cleaned_sentences), len(cleaned_sentences)])
    #print(sim_mat)
    for i in range(len(cleaned_sentences)):
        for j in range(len(cleaned_sentences)):
            if i!=j:
                sim_mat[i][j]=cosine_similarity(sentence_vectors[i].reshape(1,100),sentence_vectors[j].reshape(1,100))[0,0]
    #Ranking the each Sentence
    nx_graph = nx.from_numpy_matrix(sim_mat)
    #nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    #print(scores)
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(cleaned_sentences)), reverse=True)
    # Final output summary.....
    summary=""
    for i in range(num_sentences):
        summary_sents = ''.join(ranked_sentences[i][1])
        summary +=(summary_sents+".")
    return(summary)

In [101]:
article_text = input('https://en.wikipedia.org/wiki/Fluid_mechanics')
#print(article_text)
sentences= sent_tokens(article_text)
#print(len(sentences))
#print(sentences)
cleaned_sentences = preprocess(sentences)
print(len(cleaned_sentences))
#summary=""
Summary = re.sub(r'\s+', " ",str(word_embedded('glove.6B.100d.txt',cleaned_sentences,5)))
print(Summary)

74
solution fluid dynamics problem typically involves calculating various properties fluid velocity pressure density temperature functions space time .fluid mechanics also defined science deals study behaviour fluids either rest motion .fluid mechanics especially fluid dynamics active field research typically mathematically complex .particle image velocimetry experimental method visualizing analyzing fluid flow also takes advantage highly visual nature fluid flow .mechanical view fluid substance support shear stress fluid rest shape containing vessel .


References:
https://www.quora.com/Natural-Language-Processing-What-are-algorithms-for-auto-summarize-text
https://medium.com/@japneet121/word-vectorization-using-glove-76919685ee0b
https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
https://nlp.stanford.edu/projects/glove/